In [1]:
from transformers import pipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import tensorflow as tf
import numpy as np
import datasets

/home/student/.envs/huggingface/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'tensorflow'

In [70]:
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
model_name2 = 'autoevaluate/multi-class-classification'

classifirer = pipeline('text-classification', model=model_name)
results = classifirer([
    'Price, Good Picture Quality, Ease of use.',
    'compact size',
    'Cheap and great resolution for digital photography and graphics',
    "can't download ringtones",
    'Good. Light, affordable, just like the C80.'
    "Didn't we just buy 12 AA batteries last week????"]
    )
print(results)


[{'label': 'POSITIVE', 'score': 0.9998247027397156}, {'label': 'POSITIVE', 'score': 0.9996428489685059}, {'label': 'NEGATIVE', 'score': 0.9970999956130981}, {'label': 'NEGATIVE', 'score': 0.9918795228004456}, {'label': 'POSITIVE', 'score': 0.9986391663551331}]


In [71]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

In [72]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
# Load dataset and split into train and test

import pandas as pd
from sklearn.model_selection import train_test_split

df_train = pd.read_csv('../data/train/cr/train.txt', sep='|', header=None, names=['text'])
df_test = pd.read_csv('../data/test/cr/test.txt', sep='|', header=None, names=['text'])
df_train['class'] = df_train['text'].apply(lambda x: x.split('\t')[0])
df_train['text'] = df_train['text'].apply(lambda x: x.split('\t')[1])
df_test['class'] = df_test['text'].apply(lambda x: x.split('\t')[0])
df_test['text'] = df_test['text'].apply(lambda x: x.split('\t')[1])

df_train = df_train[['class', 'text']]
df_test = df_test[['class', 'text']]

X_test = df_test['text'].values.tolist()
y_test = df_test['class'].values.tolist()


X_train, X_val, y_train, y_val = train_test_split(df_train['text'], df_train['class'], test_size=0.2, random_state=42)
X_train = X_train.values.tolist()
y_train = y_train.values.tolist()
X_val = X_val.values.tolist()
y_val = y_val.values.tolist()

df_val = pd.DataFrame({'class': y_val, 'text': X_val})

In [74]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
val_encodings = tokenizer(X_val, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='tf')

In [75]:
df_train_ = datasets.Dataset.from_dict(df_train)
df_test_ = datasets.Dataset.from_dict(df_test)
#df_val_ = datasets.Dataset.from_dict(df_val)
dataset_final = datasets.DatasetDict({'train': df_train_, 'test': df_test_})

In [76]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset_final.map(tokenize_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [63]:
tokenized_train_dataset = tokenized_datasets["train"]
tokenized_test_dataset = tokenized_datasets["test"]

In [79]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

ImportError: cannot import name 'DefaultDataCollator' from 'transformers' (/home/student/.envs/da_project/lib/python3.8/site-packages/transformers/__init__.py)